<a href="https://colab.research.google.com/github/Porxelek/Python-Projelerim/blob/main/aims_task_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from wordcloud import WordCloud
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix, classification_report, f1_score, recall_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
from yellowbrick.classifier import PrecisionRecallCurve
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings("ignore")
plt.rcParams["figure.figsize"] = (10,6)
pd.set_option('display.max_columns', 50)

In [ ]:
# burda header ile ilgili duzenleme yapildi cunku data duzgun okunmuyordu 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df0 = pd.read_csv('/content/drive/MyDrive/Data/Comments_Demo.csv')

In [ ]:
df = df0.copy()
df.sample(13)

In [ ]:

def split_sep(x):
  if x.__contains__(","):
      return x.split(",")[0]
  elif x.__contains__("\|") :
      return x.split("\|")[0]
df["id"] = df["ID,Comments"].apply(split_sep)

In [ ]:
df["id"]

In [ ]:
def split_sep1(x):
  if x.__contains__(","):
      return x.split(",")[1]
  elif x.__contains__("\|") :
      return x.split("\|")[1]
df["Comments"] = df["ID,Comments"].apply(split_sep1)

In [ ]:
df["Comments"]

In [ ]:
df["ID,Comments"] = df["ID,Comments"].str.replace('|','__').astype("str")

In [ ]:
df["ID,Comments"] = df["ID,Comments"].str.replace(',','__')

In [ ]:
df["ID,Comments"] = df["ID,Comments"].str.split("__")

In [ ]:
df.sample(16)

In [ ]:
df["ID"] = df["ID,Comments"].apply(lambda x : x[0])

In [ ]:
df["ID"].astype("int64")
df["ID"]

In [ ]:
df["Comments"] = df["ID,Comments"].apply(lambda x : x[1])
df["Comments"]

In [ ]:
df.drop(columns="ID,Comments",inplace=True)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
nltk.download("punkt")
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
stop_words = stopwords.words('english')

for i in ["not", "no"]:
        stop_words.remove(i)

In [ ]:
def cleaning(data):
    
    #1. Tokenize
    text_tokens = word_tokenize(data.replace("'", "").lower())
    
    #2. Remove Puncs and numbers
    tokens_without_punc = [w for w in text_tokens if w.isalpha()]
    
    #3. Removing Stopwords
    tokens_without_sw = [t for t in tokens_without_punc if t not in stop_words]
    
    #4. lemma
    text_cleaned = [WordNetLemmatizer().lemmatize(t) for t in tokens_without_sw]
    
    #joining
    return " ".join(text_cleaned)

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

max_len = 0

# For every sentence...
max_token = []
for sent in df["Comments"]:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    max_token.append(len(input_ids))

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

In [ ]:
sum(np.array(max_token) <= 162) / len(max_token)

In [ ]:
!pip install utils

In [ ]:
import matplotlib
from sklearn.decomposition import PCA
from datetime import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier
#from utils import display_pca_scatterplot
import json
import pandas as pd
import random
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import f1_score, classification_report
from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish
import numpy as np
import warnings
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch

In [ ]:
train, test = \
              np.split(df["Comments"].sample(frac=1, random_state=42), 
                       [int(.9*len(df["Comments"]))])

In [ ]:
mapping = {'negative':0, 'neutral':1, 'positive':2}
X_train = []
y_train = []
X_test = []
y_test = []
for element in tqdm(train):
    X_train.append(element['sentence'])
    y_train.append(mapping[element['value']])
for element in tqdm(test):
    X_test.append(element['sentence'])
    y_test.append(mapping[element['value']])